In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
class CustomLinearRegression:
    def __init__(self, X_data, y_target, learning_rate=0.01, num_epochs=10000):
        self.num_samples = X_data.shape[0]
        self.X_data = np.c_[np.ones((self.num_samples, 1)), X_data]
        self.y_target = y_target
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.num_features = self.X_data.shape[1]

        #initialize weights
        self.theta = np.random.randn(self.X_data.shape[1], 1)
        self.losses = []

    def compute_loss(self, y_pred, y_target):
        loss = np.mean((y_pred-y_target)**2)
        return loss

    def predict(self, X_data):
        if X_data.shape[1] == self.num_features:
            y_pred = X_data.dot(self.theta)
        else:
            X_data = np.c_[np.ones((X_data.shape[0], 1)), X_data]
            y_pred = X_data.dot(self.theta)
        return y_pred

    def fit(self):
        for epoch in range (self.num_epochs):
            #predict
            y_pred = self.predict(self.X_data)
            #compute loss
            loss = self.compute_loss(y_pred, self.y_target)
            self.losses.append(loss)
            #compute gradient
            loss_grd = 2*(y_pred - self.y_target)/self.num_samples
            gradient = self.X_data.T.dot(loss_grd)
            #update weights
            self.theta -= self.learning_rate * gradient

            if (epoch % 50) == 0:
                print (f'Epoch: {epoch} - Loss: {loss}')

            return {
                'loss': sum(self.losses)/len(self.losses),
                'weight': self.theta
            }

In [19]:
def r2score(y_pred, y):
    rss = np.sum((y_pred - y)**2)
    tss = np.sum((y - y.mean())**2)
    r2 = 1 - (rss/tss)
    return r2

In [20]:
y_pred = np.array([1, 2, 3, 4, 5])
y = np.array([1, 2, 3, 4, 5])
r2score(y_pred, y)

1.0

In [21]:
y_pred = np.array([1, 2, 3, 4, 5])
y = np.array([3, 5, 5, 2, 4])
r2score(y_pred, y)

-2.235294117647059

In [22]:
def create_polynominal_features(X, degree=2):
    X_new = X
    for i in range(2, degree+1):
        X_new = np.c_[X_new, X**i]
    return X_new

In [23]:
X = np.array ([[1 , 2], 
               [2 , 3], 
               [3 , 4]])
X_new = create_polynominal_features(X, degree=2)
X_new

array([[ 1,  2,  1,  4],
       [ 2,  3,  4,  9],
       [ 3,  4,  9, 16]])

In [24]:
def create_polynominal_features(X, degree=2):
    X_mem = []
    for X_sub in X.T:
        X_sub = X_sub.T
        X_new = X_sub
        for d in range(2, degree+1):
            X_new = np.c_[X_new, np.power(X_sub, d)]
        X_mem.extend(X_new.T)
        print(X_mem)
    return np.c_[X_mem].T

In [25]:
X = np.array([[1 , 2] ,
              [2 , 3] ,
              [3 , 4]])
X_new = create_polynominal_features(X, degree=2)
X_new

[array([1, 2, 3]), array([1, 4, 9])]
[array([1, 2, 3]), array([1, 4, 9]), array([2, 3, 4]), array([ 4,  9, 16])]


array([[ 1,  1,  2,  4],
       [ 2,  4,  3,  9],
       [ 3,  9,  4, 16]])

In [26]:
df = pd.read_csv('SalesPrediction.csv')
df = pd.get_dummies(df)
df

,TV,Radio,Social Media,Sales,Influencer_Macro,Influencer_Mega,Influencer_Micro,Influencer_Nano
0,16.0,6.566231,2.907983,54.732757,False,True,False,False
1,13.0,9.237765,2.409567,46.677897,False,True,False,False
2,41.0,15.886446,2.913410,150.177829,False,True,False,False
3,83.0,30.020028,6.922304,298.246340,False,True,False,False
4,15.0,8.437408,1.405998,56.594181,False,False,True,False
...,...,...,...,...,...,...,...,...
4567,26.0,4.472360,0.717090,94.685866,False,False,True,False
4568,71.0,20.610685,6.545573,249.101915,False,False,False,True
4569,44.0,19.800072,5.096192,163.631457,False,False,True,False
4570,71.0,17.534640,1.940873,253.610411,True,False,False,False


In [27]:
df = df.fillna(df.mean())

X = df [['TV', 'Radio', 'Social Media', 'Influencer_Macro', 'Influencer_Mega', 'Influencer_Micro', 'Influencer_Nano']]
y = df [['Sales']]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train)
scaler.mean_[0]

54.173577723283785

In [29]:
X_test_processed = scaler.fit_transform(X_test)

In [30]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2)

X_train_poly = poly_features.fit_transform(X_train_processed)
X_test_poly = poly_features.fit_transform(X_test_processed)
print(X_train_poly.shape, X_test_poly.shape)
print(y_train.shape, y_test.shape)

(3063, 36) (1509, 36)
(3063, 1) (1509, 1)


In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

preds = poly_model.predict(X_test_poly)
r2_score(y_test, preds)

-8.87604557350198e+20